In [1]:
import numpy as np
from torch import nn
import torch

# 1. BatchNorm
1. 在batch内对某个channel求平均，

In [10]:
### BatchNorm2d测试

m = nn.BatchNorm1d(3, affine=False)  # affine: With Learnable Parameters or not
print('m:', m)
# The mean and std are calculated per-dimension over the mini-batches
input = torch.tensor([
    [[1.,2.,3.,4.],[1.,2.,3.,4.],[-1.,-2.,-3.,-4.]],
    [[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,0.,0.]]
], requires_grad=True)

input_mean = torch.mean(input,dim=0)
print(input_mean)
print('input:', input.shape)
output = m(input) # 归一化
print('output:', output)


m: BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
tensor([[ 0.5000,  1.0000,  1.5000,  2.0000],
        [ 0.5000,  1.0000,  1.5000,  2.0000],
        [-0.5000, -1.0000, -1.5000, -2.0000]], grad_fn=<MeanBackward1>)
input: torch.Size([2, 3, 4])
output: tensor([[[-0.1690,  0.5071,  1.1832,  1.8593],
         [-0.1690,  0.5071,  1.1832,  1.8593],
         [ 0.1690, -0.5071, -1.1832, -1.8593]],

        [[-0.8452, -0.8452, -0.8452, -0.8452],
         [-0.8452, -0.8452, -0.8452, -0.8452],
         [ 0.8452,  0.8452,  0.8452,  0.8452]]],
       grad_fn=<NativeBatchNormBackward0>)
output_2: tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3416, -0.4472,  0.4472,  1.3416],
         [ 1.3416,  0.4472, -0.4472, -1.3416]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<ViewBackward0>)


## 1.2 详细复现

In [5]:
input = torch.tensor([[[1.,2.,3.,4.]],[[0.,0.,0.,0.]]])
print(input.shape)
# torch.Size([2, 1, 4])

torch.Size([2, 1, 4])


In [6]:
BN1 = nn.BatchNorm1d(num_features=1,affine=False,eps=0)
# input只有1个feature（只有1个channel），每个features的长度=4，第一个batch
print("---BN1---")
print(torch.squeeze(BN1(input)))

---BN1---
tensor([[-0.1690,  0.5071,  1.1832,  1.8593],
        [-0.8452, -0.8452, -0.8452, -0.8452]])


In [7]:
ans = (input-torch.mean(torch.flatten(input)))/torch.sqrt(torch.var(torch.flatten(input),unbiased=False))
print(torch.squeeze(ans))

tensor([[-0.1690,  0.5071,  1.1832,  1.8593],
        [-0.8452, -0.8452, -0.8452, -0.8452]])


In [8]:
# Without Learnable Parameters
m = nn.InstanceNorm1d(100)
# With Learnable Parameters
m = nn.InstanceNorm1d(100, affine=True)
input = torch.randn(20, 100, 40)
output = m(input)

In [9]:
output

tensor([[[-1.4831e+00, -8.8348e-01,  6.8768e-01,  ..., -8.1930e-01,
           6.9095e-01, -5.3252e-01],
         [-1.2327e+00, -1.2125e+00, -7.2121e-01,  ..., -4.8107e-01,
           3.9293e-01,  8.9848e-02],
         [ 5.3689e-01,  1.3290e+00,  1.5948e-01,  ...,  8.6816e-02,
          -4.4383e-01,  1.2817e+00],
         ...,
         [-9.7964e-02, -1.9393e-01,  4.0485e-01,  ...,  1.0258e-01,
          -9.5850e-01, -1.0013e+00],
         [ 4.5442e-01,  3.6064e-01,  3.2127e-01,  ..., -1.1502e+00,
           5.7419e-01, -7.5054e-01],
         [-4.1566e-01, -2.6860e-01, -7.3793e-01,  ..., -2.7608e-01,
          -1.4282e+00,  2.0695e+00]],

        [[-2.0859e-01, -8.0312e-01, -1.6692e+00,  ...,  6.6337e-01,
          -1.7105e-01, -5.3840e-01],
         [ 1.2919e+00, -7.0192e-01,  2.2917e+00,  ...,  9.9064e-01,
          -9.4029e-02,  2.6051e-01],
         [ 3.3777e-01,  4.2390e-01,  2.9630e-01,  ...,  1.6087e+00,
           6.4012e-02, -1.1709e-02],
         ...,
         [ 8.0837e-01,  4

# 2. InstanceNorm
1. 此时只和[1.,2.,3.,4.]Norm后的结果，只和这一组数据有关，所以是“instance”

In [29]:
input = torch.tensor([
    [[1.,2.,3.,4.],[0.1,0.2,0.3,0.4],[-1.,-2.,-3.,-4.]],
    [[0.,0.,0.,0.],[0.,0.,0.,0.],[0.,0.,0.,0.]]
], requires_grad=True)

print(input.shape)
m2 = nn.InstanceNorm1d(num_features=3,affine=False)
output_2 = m2(input)
print('output_2:', output_2)

torch.Size([2, 3, 4])
output_2: tensor([[[-1.3416, -0.4472,  0.4472,  1.3416],
         [-1.3411, -0.4470,  0.4470,  1.3411],
         [ 1.3416,  0.4472, -0.4472, -1.3416]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<ViewBackward0>)


In [17]:
# 复现：
one_chanel = torch.tensor([1.,2.,3.,4.])
mean_value = torch.mean(one_chanel)
std_value = torch.std(one_chanel,unbiased=False)

(one_chanel - mean_value) / std_value

tensor([-1.3416, -0.4472,  0.4472,  1.3416])

In [37]:
input = torch.tensor([
    [1.,2.,3.,4.],[0.01,0.02,0.03,0.04],[-1.,-2.,-3.,-4.],
], requires_grad=True)
print(input.shape)

m3 = nn.InstanceNorm1d(num_features=3,affine=True)
output_3 = m3(input)
print('output_3:', output_3)

torch.Size([3, 4])
output_3: tensor([[-1.3416, -0.4472,  0.4472,  1.3416],
        [-1.2910, -0.4303,  0.4303,  1.2910],
        [ 1.3416,  0.4472, -0.4472, -1.3416]], grad_fn=<SqueezeBackward1>)


# 3. LayerNorm
1. 从参数可以看出来，是elementwise的求值
2. 没有channel之分，管你什么channel什么batch，除了`normalized_shape`，其他维度都flatten，都拿进来求mean求std
3. RNN中使用它是因为：
    - 对于同一时刻的数据，必须保持相对大小“统一”
4. https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html#layernorm

In [44]:
input = torch.tensor([
    [[0,2.,3.,4.],[1,0.2,0.3,0.4],[-1.,-2.,-3.,-4.]],
    [[-1,0.,0.,0.],[-1,0.,0.,0.],[1,0.,0.,0.]]
], requires_grad=True)

print(input.shape)
m4 = nn.LayerNorm(normalized_shape=4,elementwise_affine=True)
output_4 = m4(input)
print('output_4:', output_4)

torch.Size([2, 3, 4])
torch.Size([6, 4])
output_4: tensor([[[-1.5213, -0.1690,  0.5071,  1.1832],
         [ 1.6867, -0.8835, -0.5622, -0.2410],
         [ 1.3416,  0.4472, -0.4472, -1.3416]],

        [[-1.7320,  0.5773,  0.5773,  0.5773],
         [-1.7320,  0.5773,  0.5773,  0.5773],
         [ 1.7320, -0.5773, -0.5773, -0.5773]]],
       grad_fn=<NativeLayerNormBackward0>)


# 4. GROUPNORM
1. 思想是把channel分成不同的group：
    - 可以和InstanceNorm互换
    - 可以和LayerNorm互换

In [ ]:
# Separate 6 channels into 6 groups (equivalent with InstanceNorm)
m = nn.GroupNorm(6, 6)

#Put all 6 channels into a single group (equivalent with LayerNorm)
m = nn.GroupNorm(1, 6)

In [51]:
import numpy as np
sum(np.arange(1,8))/8

3.5

In [52]:
sum(np.arange(1,3))/6 + sum(np.arange(4,8))/10

2.7